# L4 Evaluate Inputs: Inferring & Classification

- 이 파일에서는 제품 리뷰, 뉴스 기사에서 감정과 주제를 유추하고 분류함

- LLM을 zero-shot learning을 통해 간단한 분류기로 사용하는 실습을 진행

## Setup


In [1]:
# 2024.07.31: 구글 코립일 경우 패키지 설치해야 함
!pip install -qU openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.3/389.3 kB 4.3 MB/s eta 0:00:00


In [2]:
import os
import openai

from openai import OpenAI
from google.colab import userdata

client = OpenAI(
    api_key=userdata.get('OPENAI_API_KEY')
)

#### helper function


In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    # 단일 메세지를 전달받아 message형식으로 구성
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )

    return response.choices[0].message.content


def get_completion_from_messages(messages,
                                 model="gpt-3.5-turbo",
                                 temperature=0,
                                 max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens, # 모델이 출력하는 최대 토큰
    )

    return response.choices[0].message.content


In [4]:
prompt_templete = f"""
    테스트 프롬프트입니다.

    텍스트: '''<|text|>'''
    """

# 특정 모양의 프롬프트 템플릿을 전달받아 템플릿 내부에 변수를 값으로 바꾸는 함수
def make_prompt(prompt_templete, vars, var_names=['text']):
    for var_name, var in zip(var_names, vars):
        prompt_templete = prompt_templete.replace(f"<|{var_name}|>", var)

    return prompt_templete


prompt = make_prompt(prompt_templete, ['샘플 뉴스'])
print(prompt)


    테스트 프롬프트입니다.

    텍스트: '''샘플 뉴스'''
    


## 감정 분석

- 주어진 글을 특정 감정으로 분류하기

### 리뷰 감정 분류기(다클래스 분류: 긍정적/부정적/복합적)

In [55]:
review = """
노캔과 음질은 최강입니다. 통화 품질은 별로에요
1월에 친구 생일이 있어서 선물용으로 구매했어요.
포장도 깔끔하고 뭐 배송은 로켓 배송으로
생가했던 날짜 보다 하루 빨리 도착해서 만족스럽습니다.

BOSE 이어버드2가 노이즈 캔슬링이나 음질면에서는
지금까지 나온 무선 이어폰중에는 최상급이라고 하더라구요.
그런데 이게 복불복이라서 초기 불량도 많다고 하던데 괜찮을지 모르겠네요.

불량 관련해서 BOSE코리아에서 잘 대응해준다고 하니까
믿고 쓰라고 하려구요.

저는 주로 에어팟을 사용하는데 친구는 보스 제품을 좋아하더라구요.

직접 사용해봤는데 노이즈 캔슬링일때 화이트노이즈가 좀 강하더라구요.
음악을 들으면 상관없는데 착용하고 아무것도 안키면 노이즈가 좀있습니다.
최근에 펌웨어 업데이트하니까 화이트노이즈는 확실히 줄었구요.

사운드는 진짜 최고라고 말할 수 있을것 같네요.
울리는 베이스가 차원이 다르네요.
지금 제가 소니, 애플 제품 사용중인데 비교도 안되는 수준입니다.

단점이 있다면 무선충전이 안되는거랑 주변은 허용모드를 상시로
사용하기 함들단점 그밖에 세세한 기능들이 좀 부족하다는 점이 아쉽네요.

착용감은 아주 좋습니다. 착요하고 산책을 많이가는데 이어폰을
낀건가 싶을 정도로 착용감이 가볍습니다.

마지막으로 가격은 사악합니다.
제품이 좋은것도 맞지만 가격은 너무 하네요.

구매하시면 펌웨어 업데이트 꼭 하세요.
"""

In [6]:
prompt_templete_1 = f"""
'''으로 구분된 다음 제품 리뷰의 감정은 무엇입니까?

리뷰: '''<|review|>'''
"""

In [7]:
prompt = make_prompt(prompt_templete_1, [review], ['review'])
print(prompt)


'''으로 구분된 다음 제품 리뷰의 감정은 무엇입니까?

리뷰: '''
노캔과 음질은 최강입니다. 통화 품질은 별로에요
1월에 친구 생일이 있어서 선물용으로 구매했어요.
포장도 깔끔하고 뭐 배송은 로켓 배송으로
생가했던 날짜 보다 하루 빨리 도착해서 만족스럽습니다.

BOSE 이어버드2가 노이즈 캔슬링이나 음질면에서는
지금까지 나온 무선 이어폰중에는 최상급이라고 하더라구요.
그런데 이게 복불복이라서 초기 불량도 많다고 하던데 괜찮을지 모르겠네요.

불량 관련해서 BOSE코리아에서 잘 대응해준다고 하니까
믿고 쓰라고 하려구요.

저는 주로 에어팟을 사용하는데 친구는 보스 제품을 좋아하더라구요.

직접 사용해봤는데 노이즈 캔슬링일때 화이트노이즈가 좀 강하더라구요.
음악을 들으면 상관없는데 착용하고 아무것도 안키면 노이즈가 좀있습니다.
최근에 펌웨어 업데이트하니까 화이트노이즈는 확실히 줄었구요.

사운드는 진짜 최고라고 말할 수 있을것 같네요.
울리는 베이스가 차원이 다르네요.
지금 제가 소니, 애플 제품 사용중인데 비교도 안되는 수준입니다.

단점이 있다면 무선충전이 안되는거랑 주변은 허용모드를 상시로
사용하기 함들단점 그밖에 세세한 기능들이 좀 부족하다는 점이 아쉽네요.

착용감은 아주 좋습니다. 착요하고 산책을 많이가는데 이어폰을
낀건가 싶을 정도로 착용감이 가볍습니다.

마지막으로 가격은 사악합니다.
제품이 좋은것도 맞지만 가격은 너무 하네요.

구매하시면 펌웨어 업데이트 꼭 하세요.
'''



In [8]:
response = get_completion(prompt)
print(response)

긍정적인 감정: 노이즈 캔슬링과 음질이 최고라고 말하고, 포장이 깔끔하고 배송이 빨랐다고 언급함
부정적인 감정: 통화 품질이 별로라고 언급하고, 초기 불량이 많다는 우려를 표현하며 가격이 사악하다고 언급함


In [9]:
response = get_completion(prompt, model='gpt-4o-mini')
print(response)

이 제품 리뷰의 감정은 전반적으로 긍정적입니다. 리뷰어는 제품의 노이즈 캔슬링과 음질에 대해 매우 만족하고 있으며, 착용감도 좋다고 언급하고 있습니다. 그러나 통화 품질이 별로라는 점, 무선 충전이 안 되는 점, 그리고 가격이 비싸다는 단점도 지적하고 있습니다. 전반적으로 제품의 성능에 대한 긍정적인 평가가 많지만, 몇 가지 단점도 언급되어 있어 균형 잡힌 시각을 보여줍니다.


In [10]:
prompt_templete_2 = f"""
'''으로 구분된 다음 제품 리뷰의 감정은 무엇입니까?

다음중 한 단어로 대답하세요.
"긍정적", "부정적", "복합적"

리뷰: '''<|review|>'''
"""

In [11]:
response = get_completion(
    make_prompt(prompt_templete_2, [review], ['review'])
)
print(response)

복합적


In [12]:
response = get_completion(
    make_prompt(prompt_templete_2, [review], ['review']),
    model='gpt-4o-mini'
)
print(response)

복합적


### 감정의 목록 파악

- 리뷰안에 들어있는 감정의 목록을 추출

- 뉴스의 토픽 모델링등에 사용 가능

In [13]:
prompt_templete_3 = f"""
다음 리뷰 작성자가 표현하고 있는 감정의 목록을 파악하세요.
목록에 5개 이하의 항목을 포함시키십시오.
쉼표로 구분된 단어의 목록으로 작성합니다.

리뷰: '''<|review|>'''
"""


In [14]:
response = get_completion(
    make_prompt(prompt_templete_3, [review], ['review'])
)
print(response)

만족, 불만, 의심, 놀람, 만족스러움


In [15]:
response = get_completion(
    make_prompt(prompt_templete_3, [review], ['review']),
    model='gpt-4o-mini'
)
print(response)

만족, 아쉬움, 걱정, 기쁨, 불만


### 특히 화남 리뷰의 구분(이진분류)

In [16]:
prompt_templete_4 = f"""
'''로 구분된 다음 리뷰 작성자가 분노를 표현하고 있나요?
예 또는 아니오로 답하십시오.

리뷰: '''<|review|>'''
"""


In [17]:
response = get_completion(
    make_prompt(prompt_templete_4, [review], ['review'])
)
print(response)

아니오.


In [18]:
response = get_completion(
    make_prompt(prompt_templete_4, [review], ['review']),
    model='gpt-4o-mini'
)
print(response)

아니오


### 실습

- 주어진 리뷰를 다음과 같은 형식의 json으로 출력하도록 프롬프트를 작성하시오.
```json
{
  "Item": "BOSE 이어버드2",
  "Brand": "BOSE",
  "Sentiment": "복합적",
  "Anger": "알 수 없음"
}
```

In [56]:
prompt_templete = f"""
리뷰에서 다음 항목을 식별합니다
- 리뷰어가 구매한 아이템
- 아이템을 만든 회사
- 리뷰의 감성 (긍정적, 부정적, 복합적)
- 리뷰작성자의 화남 여부 (true, false)

"Item", "Brand", "Sentiment", "Anger"를 키로 하여 응답 형식을 JSON 객체로 지정합니다.
정보가 없는 경우 값으로 "알 수 없음"을 사용합니다.
응답은 가능한 한 짧게 작성하세요.

리뷰: '''<|review|>'''
"""


In [57]:
response = get_completion(
    make_prompt(prompt_templete, [review], ['review'])
)
print(response)

{
    "Item": "무선 이어폰",
    "Brand": "BOSE",
    "Sentiment": "복합적",
    "Anger": "false"
}


In [58]:
import json

json.loads(
    response.strip("```json")
)

{'Item': '무선 이어폰', 'Brand': 'BOSE', 'Sentiment': '복합적', 'Anger': 'false'}

In [59]:
response = get_completion(
    make_prompt(prompt_templete, [review], ['review']),
    model='gpt-4o-mini'
)
print(response)

```json
{
  "Item": "무선 이어폰",
  "Brand": "BOSE",
  "Sentiment": "복합적",
  "Anger": false
}
```


In [60]:
if json.loads(response.strip("```json"))['Anger']:
    print("화남")
else:
    print("화 안남")

화 안남


## 토픽 추론하기

In [31]:
news1 = """
북한이 이틀 연속 탄도미사일을 발사한 데 대해 국가안보실은 오늘(18일) 국가안전보장회의(NSC) 상임위를 열고 대응 방안을 논의했습니다.
오늘 열린 긴급 NSC 상임위원회에는 조태용 국가안보실장을 주재로, 김대기 대통령 비서실장, 김영호 통일부 장관, 신원식 국방부 장관, 장호진 외교부 1차관, 홍장원 국가정보원 1차장, 김태효 NSC 사무처장, 인성환 국가안보실 2차장 등이 참석했습니다.
윤석열 대통령도 임석해 합참의장의 상황 보고를 받고 대응 방안을 지시했습니다.
NSC 상임위원들은 북한이 국제사회의 경고와 안보리 결의를 정면으로 무시하고 지난달 21일 소위 군사정찰위성을 발사한 데 이어 고체연료 사용 장거리 탄도미사일(ICBM)을 발사함으로써 한반도와 국제사회의 평화와 안전을 심각하게 위협하고 있음을 강력히 규탄했습니다.
또 북한이 핵무력 정책을 헌법에 명기하고 자체 핵·미사일 프로그램 개발에 몰두하면서 오히려 그 책임을 한미 동맹에 전가하고 있음에 주목했습니다. 이에 따라 정부는 우리의 안보태세를 확고히 하고, 국민의 생명과 안전을 보장하기 위한 조치를 계속 취해 나가야 함을 강조했습니다.
윤석열 대통령은 우리 영토와 국민에 대한 북한의 어떠한 도발도 즉시, 압도적으로 대응할 것을 지시했습니다. 이를 위해 한미 연합방위태세를 굳건하게 유지하고, 북한 미사일 경보 정보 실시간 공유 체계를 활용하여 한미일의 공동대응을 적극 추진할 것을 지시했습니다.
그러면서 윤 대통령은 국제사회와 적극 연대해 북한의 불법적인 핵·미사일 활동을 규탄하고 저지해 나갈 것을 지시했습니다. 나아가 한미 핵협의그룹(NCG)의 과제도 속도감 있게 추진해 한미의 대북 핵 억제 실행력을 더욱 강화할 것을 지시했습니다. 아울러 북한의 연말연시 추가 도발 가능성에 대비하여 대응태세에 만전을 기할 것을 강조했습니다.
"""

news2 = """
지난해 혼인 신고를 한 지 5년이 안 된 신혼부부는 81만5천3백여 쌍,
이 가운데 맞벌이를 하면서 자녀가 없는 이른바 딩크족 부부는 23만4천여 쌍으로 28.7%를 차지해 가장 많았습니다.
맞벌이를 하면서 자녀를 키우는 부부가 28.5%로 뒤를 이었고, 외벌이에 자녀를 키우는 부부가 22.7%, 외벌이·무자녀 부부는 15.5% 순이었습니다.
2015년 신혼부부 가운데 18%였던 '딩크족' 비중은 점점 늘어 지난 2021년 맞벌이·유자녀 비중을 처음 넘어섰습니다.
이런 현상은 여성의 경제활동 참가가 늘어나고, 아이를 낳지 않아도 괜찮다는 인식이 확산한 결과로 보입니다.
여성가족부 사회실태조사 결과를 보면 20~30대 청년층에서 무자녀에 대한 긍정적 인식이 2015년 27.7%에서 2020년 44.1%로 높아졌습니다.
자녀를 가져도 첫째 아이를 낳는 평균 기간이 2.7년으로 출산 시기는 점점 늦춰지고 있습니다.
그 결과 결혼 생활이 5년 이상일 때 첫째 아이를 낳은 비중이 지난해 11.5%로 10년 만에 2배로 늘었습니다.
맞벌이로 소득은 높지만 자녀를 갖지 않는 부부가 늘면서, 초혼 신혼부부의 경우 소득이 높을수록 평균 자녀 수가 감소하고 있습니다.
연 소득 1억 원 이상 초혼 신혼 부부의 평균 자녀 수는 0.57명으로 연소득 천만 원 미만인 경우의 0.77명보다도 적었습니다.
"""

news3 = """
정부는 소비자 물가 부담을 줄이고, 수입산 증류주와 형평성을 맞추기 위해 기준판매비율을 도입했습니다.
세금을 계산할 때 일정 비율을 할인해 주는 개념인데, 소주는 22%, 위스키는 23.9%로 정했습니다.
이걸 적용하면 내년부터 소주 한병의 공장 출고가는 지금보다 132원, 10.6% 내립니다.
소비자 입장에선 반가운 소식입니다.
[이태호/서울 불광동 : 소주 한 병에 7000원까지 올라가는 게 좀 비싸다 생각을 했는데 내려간다 그러면 사 먹는 입장에서는 좋지 않을까.]
다만 주류 제조사들이 이미 출고가를 올렸거나, 올릴 예정이라 세금 인하 효과가 온전히 나타나긴 어려워 보입니다.
하이트진로는 지난달 이미 참이슬 출고가를 7% 올렸고, 롯데칠성 역시 내년부터 처음처럼 반출가격을 7% 가까이 올립니다.
'새로'는 8.9% 인상합니다.
소주 주원료인 주정이나 병값이 올랐단 이유입니다.
깎인 세금을 고려하면 마트나 편의점에선 가격이 소폭 내릴 수 있지만, 이미 7천원대까지 오른 음식점이나 술집은 또 다릅니다.
[A씨/음식점 운영 : 공급가가 떨어지면 공급가가 떨어진 만큼 술값을 내릴 의향은 있는데 그것 역시도 저희가 빨리빨리 가격을 반영하지 않는 스타일로 장사하기 때문에…]
생소비자 피부에 와닿을 정도로 가격은 내려가지 못하고 세수만 더 줄어들 수 있다는 비판도 나옵니다.
"""



### 토픽 추출

In [32]:
prompt_templete = f"""
다음 '''으로 구분된 텍스트에서 논의되고 있는 세 개의 주제를 결정합니다.
각 주제 항목은 서너 단어 길이로 작성합니다.
쉼표로 구분된 항목 목록으로 응답 형식을 지정합니다.

텍스트: '''<|news|>'''
"""


In [33]:
response = get_completion(
    make_prompt(prompt_templete, vars=[news1], var_names=['news']),
)
print(response)

국가안보실 대응 방안 논의
NSC 상임위원회 참석자
북한의 도발 대응 방침


In [34]:
response = get_completion(
    make_prompt(prompt_templete, vars=[news1], var_names=['news']),
    model='gpt-4o-mini'
)
print(response)

북한 미사일 발사, 국가안보회의 대응, 한미 연합 방위태세


In [35]:
response = get_completion(
    make_prompt(prompt_templete, vars=[news2], var_names=['news']),
    model='gpt-4o-mini'
)
print(response)

딩크족 증가, 맞벌이 부부, 자녀 출산 지연


In [36]:
response = get_completion(
    make_prompt(prompt_templete, vars=[news3], var_names=['news']),
    model='gpt-4o-mini'
)
print(response)

소비자 물가 부담, 주류 세금 인하, 주류 가격 인상


### 토픽 분류

In [37]:
topic_list = [
    "남북", "핵무기", "인구", "경제", "민생"
]

prompt_templete = f"""'''으로 구분된 텍스트의 주제가
다음 주어진 주제 목록의 각 항목에 해당하는지 여부를 결정하십시오.

각 주제에 대해 0 또는 1이 포함된 목록으로 답하십시오.
형식: 주제 이름: 0 또는 1

주제 목록: {", ".join(topic_list)}

텍스트: '''<|story|>'''
"""



In [39]:
print(make_prompt(prompt_templete, vars=[news1], var_names=['story']))

'''으로 구분된 텍스트의 주제가
다음 주어진 주제 목록의 각 항목에 해당하는지 여부를 결정하십시오.

각 주제에 대해 0 또는 1이 포함된 목록으로 답하십시오.
형식: 주제 이름: 0 또는 1

주제 목록: 남북, 핵무기, 인구, 경제, 민생

텍스트: '''
북한이 이틀 연속 탄도미사일을 발사한 데 대해 국가안보실은 오늘(18일) 국가안전보장회의(NSC) 상임위를 열고 대응 방안을 논의했습니다.
오늘 열린 긴급 NSC 상임위원회에는 조태용 국가안보실장을 주재로, 김대기 대통령 비서실장, 김영호 통일부 장관, 신원식 국방부 장관, 장호진 외교부 1차관, 홍장원 국가정보원 1차장, 김태효 NSC 사무처장, 인성환 국가안보실 2차장 등이 참석했습니다.
윤석열 대통령도 임석해 합참의장의 상황 보고를 받고 대응 방안을 지시했습니다.
NSC 상임위원들은 북한이 국제사회의 경고와 안보리 결의를 정면으로 무시하고 지난달 21일 소위 군사정찰위성을 발사한 데 이어 고체연료 사용 장거리 탄도미사일(ICBM)을 발사함으로써 한반도와 국제사회의 평화와 안전을 심각하게 위협하고 있음을 강력히 규탄했습니다.
또 북한이 핵무력 정책을 헌법에 명기하고 자체 핵·미사일 프로그램 개발에 몰두하면서 오히려 그 책임을 한미 동맹에 전가하고 있음에 주목했습니다. 이에 따라 정부는 우리의 안보태세를 확고히 하고, 국민의 생명과 안전을 보장하기 위한 조치를 계속 취해 나가야 함을 강조했습니다.
윤석열 대통령은 우리 영토와 국민에 대한 북한의 어떠한 도발도 즉시, 압도적으로 대응할 것을 지시했습니다. 이를 위해 한미 연합방위태세를 굳건하게 유지하고, 북한 미사일 경보 정보 실시간 공유 체계를 활용하여 한미일의 공동대응을 적극 추진할 것을 지시했습니다.
그러면서 윤 대통령은 국제사회와 적극 연대해 북한의 불법적인 핵·미사일 활동을 규탄하고 저지해 나갈 것을 지시했습니다. 나아가 한미 핵협의그룹(NCG)의 과제도 속도감 있게 추진해 한미의 대북 핵 억제 실행력을 더욱 강화할 것을 지시했습니다. 아울러 

In [44]:
response = get_completion(
    make_prompt(prompt_templete, vars=[news1], var_names=['story']),
    model='gpt-4o-mini'
)
print(response)

남북: 1  
핵무기: 1  
인구: 0  
경제: 0  
민생: 0  


In [41]:
response = get_completion(
    make_prompt(prompt_templete, vars=[news2], var_names=['story']),
    model='gpt-4o-mini'
)
print(response)

남북: 0  
핵무기: 0  
인구: 1  
경제: 1  
민생: 1  


In [42]:
response = get_completion(
    make_prompt(prompt_templete, vars=[news3], var_names=['story']),
    model='gpt-4o-mini'
)
print(response)

남북: 0  
핵무기: 0  
인구: 0  
경제: 1  
민생: 1  


In [45]:
# 특정 토픽의 뉴스가 보도되면 자동으로 판단하기

# 모델의 응답을 사전으로 변환
topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n')}

# 사전에 핵무기 키에 값이 1이상이면 알림
if topic_dict['핵무기'] == 1:
    print("ALERT: 핵무기 뉴스!")

ALERT: 핵무기 뉴스!


## 고객의 문의를 적절한 카테고리로 분류 하기: Zero Shot 러닝

- 다음은 고객의 문의를 적절한 카테고리로 분류하는 프롬프트에 대한 예시

- 고객문의에 대한 파인튜닝 작업 없이 문의를 대분류 4개, 각 대분류별 중분류 3~4개로 분류

- 모델은 처음 보는 고객문의를 시스템 메시지에 지정한 카테고리로 분류 하기 때문에 제로샷 러닝에 해당

- 문의를 분류 하고 분류에 따라 적합하게 따로 개발된 프롬프트로 고객 응대를 하면 LLM 답변 품질, 정확성을 높일 수 있음

- 고객 문의에 대한 별도의 학습 과정없이 문의를 기본 분류 4개, 각 기본 분류별 하위 분류 3~4개로 분류

- 다음은 기본 카테고리와 보조카테고리를 나타낸 것

- 요금청구
    - 구독 취소 또는 업그레이드
    - 결제 방법 추가
    - 청구에 대한 설명
    - 청구에 대한 이의 제기

- 기술지원
    - 일반적인 문제 해결
    - 장치 호환성
    - 소프트웨어 업데이트

- 계정관리
    - 비밀번호 재설정
    - 개인 정보 업데이트
    - 계정 폐쇄
    - 계정 보안

- 일반문의
    - 제품 정보
    - 가격 문의
    - 상담원 연결



- 아래와 같은 사용자 문의에 대해서 예시처럼 기본, 보조 카테고리를 알려주면 JSON 형식의 응답을 만드는 프롬프트를 작성하시오.

- 프롬프트에 포함될 내용이 많으므로 system 메세지와 user 메세지를 구분하여 구성하는 방법을 추천

- 예시
    - 사용자 문의: 아이디의 비번을 재설정하려면 어떻게 하나요?
```json
{
  "기본 카테고리": "계정관리",
  "보조 카테고리": "비밀번호 재설정"
}
```

In [46]:
delimiter = "####"

system_message = f"""
고객 서비스 문의 사항이 제공됩니다.
고객 서비스 문의는 {delimiter} 문자로 구분됩니다.
각 문의를 기본 카테고리와 보조 카테고리로 분류합니다.
기본 및 보조 키가 포함된 json 형식의 출력을 제공해야 합니다.

기본 카테고리: 요금청구, 기술지원, 계정관리 또는 일반문의.

요금청구에 대한 보조 카테고리:
구독 취소 또는 업그레이드
결제 방법 추가
청구에 대한 설명
청구에 대한 이의 제기

기술지원에 대한 보조 카테고리
일반적인 문제 해결
장치 호환성
소프트웨어 업데이트

계정관리에 대한 보조 카테고리
비밀번호 재설정
개인 정보 업데이트
계정 폐쇄
계정 보안

일반문의에 대한 보조 카테고리:
제품 정보
가격 문의
상담원 연결
"""



In [49]:
user_message = f"""내 프로필과 사용자 데이터를 모두 삭제하고 싶습니다."""
# user_message = f"""아이디의 비번을 재설정하려면 어떻게 하나요?"""

messages =  [
    {
        'role':'system',
        'content': system_message
    },
    {
        'role':'user',
        'content': f"{delimiter}{user_message}{delimiter}"
    },
]

response = get_completion_from_messages(messages)
print(response)

{
  "기본 카테고리": "계정관리",
  "보조 카테고리": "계정 폐쇄"
}


In [52]:
user_message = f"""평면 스크린 TV에 대해서 알고 싶습니다."""

messages =  [
    {
        'role':'system',
        'content': system_message
    },
    {
        'role':'user',
        'content': f"{delimiter}{user_message}{delimiter}"
    },
]

response = get_completion_from_messages(messages)
print(response)

```json
{
    "기본 카테고리": "일반문의",
    "보조 카테고리": "제품 정보"
}
```


In [53]:
user_message = f"""RTX 4090 요즘 가격 문의에 대한 응답을 언제쯤 받을 수 있나요?"""

messages =  [
    {
        'role':'system',
        'content': system_message
    },
    {
        'role':'user',
        'content': f"{delimiter}{user_message}{delimiter}"
    },
]

response = get_completion_from_messages(messages)
print(response)

```json
{
    "기본 카테고리": "일반문의",
    "보조 카테고리": "가격 문의"
}
```


In [54]:
user_message = f"""상담원과 통화하고 싶어요."""

messages =  [
    {
        'role':'system',
        'content': system_message
    },
    {
        'role':'user',
        'content': f"{delimiter}{user_message}{delimiter}"
    },
]

response = get_completion_from_messages(messages)
print(response)

{
  "기본 카테고리": "일반문의",
  "보조 카테고리": "상담원 연결"
}


- 이렇게 문의를 먼저 분류한 뒤 각 분류에 따라 별도로 개발된 프롬프트를 사용해 고객 문의에 대한 응답을 생성

In [ ]:
# 여기 실습 과제 하나 만들기
